# Test the performance of the model

## Occurrence of symmetry equivalent structures

### Binary Quadratic Model

In [2]:
from AQC_CMC_workflow import *

In [3]:
lattice = np.array([[ 1.233862, -2.137112,  0.      ],
                   [ 1.233862,  2.137112,  0.      ],
                   [ 0.      ,  0.      ,  8.685038]])

graphene = Structure(lattice, species=['C','C'], coords=[[2/3, 1/3, 0. ],[1/3, 2/3, 0.]])
graphene = SpacegroupAnalyzer(graphene).get_conventional_standard_structure()

n_supercell = 3
scaling_matrix = np.identity(3)*n_supercell
scaling_matrix[2][2] = 1
graphene_supercell = copy.deepcopy(graphene)
graphene_supercell.make_supercell(scaling_matrix)

In [4]:
structure = graphene_supercell
use_coord = False
num_vacancies=1
weight_1 = 1
weight_2 = 0
lagrange = 10
num_reads = 1000
file_path = 'data/test_symmetry'
file_name = 'graphene'
remove_broken_chains = False

In [5]:
bqm = build_quadratic_model(structure,use_coord=use_coord,weight_1=weight_1,weight_2=weight_2,
                             lagrange=lagrange, num_vacancies=num_vacancies)



Unconstrained quadratic model + contraints used


In [12]:
dataframe, config, energy = run_anneal(bqm,num_reads=num_reads, return_config_E=True,remove_broken_chains=remove_broken_chains)

In [79]:
#IMPORTED
def ratio_broken_chains(dataframe):
    # Return how many solutions returned a broken chain
    
    broken = np.sum(dataframe[dataframe['chain_break_fraction'] == 0.]['num_occurrences'].to_numpy())
    total = np.sum(dataframe['num_occurrences'].to_numpy())
    
    return np.round(broken/total,4)

In [82]:
def energy_distribution(dataframe,remove_broken_chains = True):
    
    if remove_broken_chains == True:
        df = dataframe[dataframe['chain_break_fraction'] == 0.]
    elif remove_broken_chains == False:
        df = dataframe
        
    energy = df['energy']
    energy = np.round(np.array(energy),5)
    multiplicity = df['num_occurrences']
    multiplicity = np.array(multiplicity)

    unique_multiplicity = []

    unique_energy = np.unique(energy)
    for e in unique_energy:
        pos = np.where(energy == np.round(e,5))[0]
        unique_multiplicity.append(np.sum(multiplicity[pos]))
    
    return unique_energy, unique_multiplicity

In [41]:
def ratio_feasible(dataframe,num_vacancies, remove_broken_chains = True):
    
    if remove_broken_chains == True:
        df = dataframe[dataframe['chain_break_fraction'] == 0.]
    elif remove_broken_chains == False:
        df = dataframe
       
    num_atoms = sum([x.isdigit() for x in df.columns])
    
    all_config = df.iloc[:,0:num_atoms].to_numpy()

    multiplicity = df['num_occurrences'].to_numpy()

    sum_vector = np.sum(all_config,axis=1)

    feasible_config = np.where(np.round(sum_vector,5) == np.round((num_atoms - num_vacancies),5))[0]

    total_feasible = np.sum(multiplicity[feasible_config])

    ratio_feasible = total_feasible/np.sum(multiplicity)
    
    return np.round(ratio_feasible,4)
    
    

In [40]:
sum([x.isdigit() for x in df.columns])

18

In [81]:
def equivalent_energy_distribution(dataframe, energy, remove_broken_chains = True, sort_config = True):
    
    #Only tested for 1 vacancy
    
    if remove_broken_chains == True:
        df = dataframe[dataframe['chain_break_fraction'] == 0.]
    elif remove_broken_chains == False:
        df = dataframe
        
    energies = df['energy']
    energies = np.round(np.array(energies),6)
    
    all_config = df.iloc[:,0:n_atoms].to_numpy()
    all_multiplicity = df['num_occurrences'].to_numpy()

    config_pos = np.where(energies == np.round(energy,6))[0]
    configurations = all_config[config_pos]

    multiplicity = all_multiplicity[config_pos]
    
    if sort_config == True:
        config_sorted = []
        for i, conf in enumerate(configurations):
            config_sorted.append(int(''.join([str(x) for x in conf])))

        sorting = np.argsort(np.array(config_sorted))   
 
        config = configurations[sorting]
        mult = multiplicity[sorting]
    
        return config, mult
    
    elif sort_config == False:
        
        return configurations, multiplicity
    

In [10]:
def lowest_energy_found(dataframe, bqm):
    
    dataframe_low_e = np.min(dataframe['energy'].to_numpy())
    exact_low_e = np.min(find_exact_solutions(bqm)['energy'].to_numpy())
    
    if dataframe_low_e == exact_low_e:
        return True
    else:
        return False

In [14]:
save_json(structure,dataframe,bqm,use_coord=use_coord, num_vacancies=num_vacancies, 
          weight_1=weight_1, weight_2=weight_2, lagrange=lagrange, num_reads=num_reads,
          remove_broken_chains = remove_broken_chains,
          file_path=file_path, file_name=file_name)

In [28]:
df, param, qubo_matrix = load_json('./data/test_symmetry/graphene_C18_bqm_v1_cF_w11_w20_l10_r100_t0.json')

### Tests:
- chain_break_fracion, number of energy values, number of occurences (1v) 
- vs
- num_cycles, chain strength, cell size

# Loop

In [116]:
for lagrange in [1,10]:    
    structure = graphene_supercell
    use_coord = False
    num_vacancies=1
    weight_1 = 1
    weight_2 = 0
    lagrange = lagrange
    num_reads = 1000
    file_path = 'data/'
    file_name = 'graphene'
    remove_broken_chains = False

    bqm = build_quadratic_model(structure,use_coord=use_coord,weight_1=weight_1,weight_2=weight_2,
                                 lagrange=lagrange, num_vacancies=num_vacancies)
    
    dataframe, config, energy = run_anneal(bqm,num_reads=num_reads, return_config_E=True,remove_broken_chains=remove_broken_chains)
    
    save_json(structure,dataframe,bqm,use_coord=use_coord, num_vacancies=num_vacancies, 
          weight_1=weight_1, weight_2=weight_2, lagrange=lagrange, num_reads=num_reads,
          remove_broken_chains = remove_broken_chains,
          file_path=file_path, file_name=file_name)
    
    

Unconstrained quadratic model + contraints used
Unconstrained quadratic model + contraints used


In [84]:
import os

directory = 'data'

dataframes = []

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and f.endswith(".json"):
        print(f)
        df, param = load_json(f,return_param = True, return_qubo = False)
        param = param.transpose()
        
        param['ratio_feasible'] = ratio_feasible(df,param['num_vacancies'].values[0])
        param['broken_chains'] = ratio_broken_chains(df)
        param['energy_distribution'] = ', '.join(str(x) for x in energy_distribution(df)[0])
        param['multiplicity'] = ', '.join(str(x) for x in energy_distribution(df)[1])
        
        dataframes.append(param)
pd.concat(dataframes)

data/graphene_C18_bqm_v1_cF_w11_w20_l1_r1000_t0.json
data/graphene_C18_bqm_v1_cF_w11_w20_l10_r1000_t0.json


,date_time,structure,N atoms,model,use_coord,num_vacancies,weight_1,weight_2,lagrange,num_reads,time_limit,label,remove_broken_chains,ratio_feasible,broken_chains,energy_distribution,multiplicity
Values,"07/05/2022, 17:49:03",C18,18,bqm,False,1,1,0,1,1000,0,Test anneal,False,0.5385,0.975,"-26.0, -24.0, -21.0, -20.0","406, 525, 18, 26"
Values,"07/05/2022, 17:49:11",C18,18,bqm,False,1,1,0,10,1000,0,Test anneal,False,0.3074,0.937,"-24.0, -17.0, -12.0, -11.0, 20.0, 21.0, 22.0, ...","288, 2, 138, 445, 3, 27, 32, 2"


In [20]:
b.transpose()

,date_time,structure,N atoms,model,use_coord,num_vacancies,weight_1,weight_2,lagrange,num_reads,time_limit,label,remove_broken_chains
Values,"07/05/2022, 17:49:11",C18,18,bqm,False,1,1,0,10,1000,0,Test anneal,False


In [51]:
a,b = load_json('data/graphene_C18_bqm_v1_cF_w11_w20_l1_r1000_t0.json',return_param = True, return_qubo = False)
c,d = load_json('data/graphene_C18_bqm_v1_cF_w11_w20_l10_r1000_t0.json',return_param = True, return_qubo = False)

In [66]:
e = pd.concat([b.transpose(),d.transpose()])
e['ratio'] = [','.join(str(x) for x in energy_distribution(a)[0]),','.join(str(x) for x in energy_distribution(a)[0])]
e.sort_values(by=['lagrange'])

,date_time,structure,N atoms,model,use_coord,num_vacancies,weight_1,weight_2,lagrange,num_reads,time_limit,label,remove_broken_chains,ratio
Values,"07/05/2022, 17:49:03",C18,18,bqm,False,1,1,0,1,1000,0,Test anneal,False,"-26.0,-24.0,-21.0,-20.0"
Values,"07/05/2022, 17:49:11",C18,18,bqm,False,1,1,0,10,1000,0,Test anneal,False,"-26.0,-24.0,-21.0,-20.0"


In [42]:
ratio_feasible(a,1)

0.3074

In [50]:
','.join(str(x) for x in energy_distribution(a)[0])

'-24.0,-17.0,-12.0,-11.0,20.0,21.0,22.0,74.0'

## Lagrange

In [141]:
df = load_json('data/graphene_C18_bqm_v1_cF_w11_w20_l10_r1000_t0.json',return_param = False, return_qubo = False)

In [142]:
print(ratio_broken_chains(df))
print(ratio_feasible(df,1))

0.937
0.9061


## Chain strength

In [18]:
config_all, energy_all, multiplicity_all = find_all_structures(dataframe, sort_config=True)

In [20]:
np.sum(multiplicity_all)

294

## Chain strength

## Exact solver